# Construir un Chatbot

## Descripción general
Repasaremos un ejemplo de cómo diseñar e implementar un chatbot con tecnología LLM. Este chatbot podrá mantener una conversación y recordar interacciones previas con un modelo de chat.

Tenga en cuenta que este chatbot que construimos solo usará el modelo de lenguaje para mantener una conversación.

## 📦 Instalación de dependencias necesarias

Antes de ejecutar este notebook, asegúrate de instalar las siguientes bibliotecas:

```bash
pip install langchain langchain_core langgraph typing-extensions transformers
```


In [3]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages
from typing import Sequence
from typing_extensions import Annotated, TypedDict

## Inicio rápido
Primero, aprendamos a usar un modelo de lenguaje por sí solo. LangChain admite muchos modelos de lenguaje diferentes que puedes usar indistintamente. En este notebook usaremos Ollama.

In [4]:
# Model
model = ChatOllama(
    # Modelo descargado desde Ollama (puedes cambiar el nombre según el modelo disponible)
    model="llama3.2",
    base_url="http://localhost:11434"  # URL local donde corre el servidor de Ollama
)

Primero, usemos el modelo directamente. Los ChatModels son instancias de los "Runnables" de LangChain, lo que significa que exponen una interfaz estándar para interactuar con ellos. Para llamar al modelo, podemos pasar una lista de mensajes al método .invoke.

In [8]:
# Calling the model
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello Bob! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-10T05:29:30.2203278Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3087216200, 'load_duration': 34353000, 'prompt_eval_count': 30, 'prompt_eval_duration': 122001600, 'eval_count': 27, 'eval_duration': 2930861600, 'model_name': 'llama3.2'}, id='run-0ddb2355-1786-4228-a874-2e5f3436cf05-0', usage_metadata={'input_tokens': 30, 'output_tokens': 27, 'total_tokens': 57})

El modelo por sí solo no tiene concepto de estado. Por ejemplo, si se plantea una pregunta complementaria:

In [11]:
# Model without state
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I don't have any information about your identity. I'm a large language model, I don't retain personal data or track individual users. Each time you interact with me, it's a new conversation and I don't have any prior knowledge about you. Would you like to introduce yourself?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-10T05:32:54.5515372Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7975182800, 'load_duration': 45290200, 'prompt_eval_count': 30, 'prompt_eval_duration': 675378700, 'eval_count': 59, 'eval_duration': 7253896900, 'model_name': 'llama3.2'}, id='run-37a461cc-4d3a-472f-9d91-c123d5267c89-0', usage_metadata={'input_tokens': 30, 'output_tokens': 59, 'total_tokens': 89})

Vemos que no toma en contexto la conversación anterior y no puede responder a la pregunta. Esto genera una experiencia de chatbot pésima.

Para solucionar esto, necesitamos pasar todo el historial de conversaciones al modelo. Veamos qué sucede al hacerlo:

In [12]:
# Giving the context
model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content="Your name is Bob. That's the name we established at the beginning of our conversation.", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-05-10T05:33:02.5549638Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5136282500, 'load_duration': 51295500, 'prompt_eval_count': 55, 'prompt_eval_duration': 2743480800, 'eval_count': 19, 'eval_duration': 2332604800, 'model_name': 'llama3.2'}, id='run-509088b9-4aa7-40e3-ae99-861a3fee76bf-0', usage_metadata={'input_tokens': 55, 'output_tokens': 19, 'total_tokens': 74})

## Persistencia de mensajes

LangGraph implementa una capa de persistencia integrada, lo que lo hace ideal para aplicaciones de chat que admiten múltiples turnos de conversación.

Al integrar nuestro modelo de chat en una aplicación LangGraph minimalista, podemos persistir automáticamente el historial de mensajes, simplificando así el desarrollo de aplicaciones multiturno.

In [13]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Ahora necesitamos crear una configuración que pasemos al ejecutable cada vez. Esta configuración contiene información que no forma parte de la entrada directa, pero que sigue siendo útil. En este caso, queremos incluir un thread_id. Debería verse así:

In [14]:
# Creating the thread
config = {"configurable": {"thread_id": "abc123"}}

Esto nos permite gestionar múltiples hilos de conversación con una sola aplicación, un requisito común cuando la aplicación tiene varios usuarios.

Podemos entonces invocar la aplicación:

In [15]:
# New query
query = "Hi! I'm Bob."

# input uses the new query
input_messages = [HumanMessage(query)]

# invoke the workflow compile using the input and the thread
output = app.invoke({"messages": input_messages}, config)

print(
    f"\nConversación con el thread id: {config['configurable']['thread_id']}\n")

# Printing the last message of the state
output["messages"][-1].pretty_print()  # output contains all messages in state


Conversación con el thread id: abc123

================================== Ai Message ==================================

Hello Bob! It's nice to meet you. Is there something I can help you with, or would you like to chat?


In [16]:
# Following the thread
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. You told me that earlier when we started chatting.


¡Genial! Nuestro chatbot ahora recuerda información sobre nosotros. Si cambiamos la configuración para que haga referencia a un thread_id diferente, podemos ver que inicia la conversación desde cero.

In [17]:
# Using a new thread
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

print(
    f"\nConversación con el thread id: {config['configurable']['thread_id']}\n")
output["messages"][-1].pretty_print()


Conversación con el thread id: abc234

================================== Ai Message ==================================

I don't know your name. I'm a large language model, I don't have the ability to retain information about individual users or keep track of personal identities. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations.

If you'd like to share your name with me, I'd be happy to chat with you!


Sin embargo, siempre podemos volver a la conversación original (ya que la estamos conservando en una base de datos).

In [18]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

print(
    f"\nConversación con el thread id: {config['configurable']['thread_id']}\n")
output["messages"][-1].pretty_print()


Conversación con el thread id: abc123

================================== Ai Message ==================================

You mentioned your name earlier as "Bob". However, I don't have any information about you beyond our conversation starting point. If you'd like to share more about yourself or change the subject, I'm here to help!


¡Así es como podemos ayudar a un chatbot a mantener conversaciones con muchos usuarios!

## Llamadas asíncronas
Para obtener soporte asincrónico, actualice el nodo call_model para que sea una función asincrónica y use .ainvoke al invocar la aplicación:
```Python
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()
```

Por ahora, solo hemos añadido una capa de persistencia simple alrededor del modelo. Podemos empezar a hacer el chatbot más complejo y personalizado añadiendo una plantilla de solicitud.

## Prompt Templates

Las plantillas de avisos (Prompt Templates) ayudan a convertir la información sin procesar del usuario a un formato compatible con el LLM. En este caso, la entrada sin procesar del usuario es simplemente un mensaje que pasamos al LLM. Ahora, compliquemos un poco más el proceso. Primero, añadiremos un mensaje del sistema con algunas instrucciones personalizadas (pero que seguirá aceptando mensajes como entrada). A continuación, añadiremos más información además de los mensajes.

Para añadir un mensaje del sistema, crearemos una plantilla ChatPromptTemplate. Utilizaremos MessagesPlaceholder para pasar todos los mensajes.

In [19]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",  # sytem prompt
        ),
        # from state(MessageState) we use the messages to insert as the placeholder, the MessageState automatically will add any other message
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Ahora podemos actualizar nuestra aplicación para incorporar esta plantilla:

In [23]:
# Create the new workflow with the MessagesState schema
builder = StateGraph(state_schema=MessagesState)

# New function addgin the state
def call_model_template(state: MessagesState):
    # the prompt template will invoke the messages from the MessagesState schema
    prompt = prompt_template.invoke(state)
    # the model will invoke the prompt with the full conversation
    response = model.invoke(prompt)
    # updating the messages from MessagesState with the new response
    return {"messages": response}

# Flow of the graph
builder.add_edge(START, "model_template")
# Node of the flow and the function called
builder.add_node("model_template", call_model_template)
# Memory of the flow
memory = MemorySaver()
app = builder.compile(checkpointer=memory)

Invocamos la aplicación de la misma manera:

In [24]:

config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
print(
    f"\nConversación con el thread id: {config['configurable']['thread_id']}\n")
output["messages"][-1].pretty_print()


Conversación con el thread id: abc345

================================== Ai Message ==================================

Yer lookin' fer some scurvy-fightin' chat, eh? Well, matey Jim, welcome aboard! Ol' Blackbeak be here ta help ye navigate any waters o' knowledge or trouble that come yer way. What be bringin' ye to these fair shores today?


In [25]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, me hearty Jim... *checks me trusty logbook* ...ye be known as... Jim! Aye, that be right! Ye don't need ta worry about forgettin' yer own name, matey. Now, what be ye lookin' fer knowledge about today?


¡Genial! Ahora compliquemos un poco más nuestra propuesta. Supongamos que la plantilla de propuesta ahora luce así:

In [26]:
# Complicate Prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [27]:
# Creating the new state class with TypedDict schema: use messages as an Annotaded List with a
# Sequence of BaseMessage using the method add_messages: Merges two lists of messages, updating existing messages by ID.
# Use the variable language to be use it in the prompt template
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]    # List of messages
    language: str                                               # lenguage variable


# New workflow using the State class as schema
flow = StateGraph(state_schema=State)


def call_model_complicated(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


flow.add_edge(START, "model_complicated")
flow.add_node("model_complicated", call_model_complicated)

memory = MemorySaver()
app = flow.compile(checkpointer=memory)

In [28]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
# Defining the language
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(  # invoke the app with the two variables used in the schema and the thread
    {"messages": input_messages,
     "language": language},
    config,
)
print(
    f"\nConversación con el thread id: {config['configurable']['thread_id']}\n")
output["messages"][-1].pretty_print()


Conversación con el thread id: abc456

================================== Ai Message ==================================

Hola, Bob! (Hello, Bob!) ¿En qué puedo ayudarte hoy? (How can I help you today?)


Tenga en cuenta que se conserva todo el estado, por lo que podemos omitir parámetros como el idioma si no deseamos realizar cambios:

In [29]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob. (Your name is Bob.) ¿Quieres saber algo más sobre ti mismo, Bob? (Do you want to know something more about yourself, Bob?)


## Gestión del historial de conversaciones

Un concepto importante al crear chatbots es cómo gestionar el historial de conversaciones. Si no se gestiona, la lista de mensajes crecerá sin límites y podría desbordar la ventana de contexto del LLM. Por lo tanto, es fundamental añadir un paso que limite el tamaño de los mensajes que se pasan.

Es importante hacer esto ANTES de la plantilla de solicitud, pero DESPUÉS de cargar los mensajes anteriores del Historial de mensajes.

Podemos lograrlo añadiendo un paso sencillo delante de la solicitud que modifique la clave de mensajes correctamente y, a continuación, envuelva esa nueva cadena en la clase Historial de mensajes.

LangChain incluye varios ayudantes integrados para gestionar la lista de mensajes. En este caso, usaremos el ayudante trim_messages para reducir la cantidad de mensajes que enviamos al modelo. El ayudante trim_messages nos permite especificar cuántos tokens queremos conservar, junto con otros parámetros, como si queremos conservar siempre el mensaje del sistema y si se permiten mensajes parciales.

In [39]:
from langchain_core.messages.utils import count_tokens_approximately

In [40]:
# Managing Conversation History
# Use trim_messages to reduce how many messages we're sending to the model.
# use transformers
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=count_tokens_approximately,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

# List of messages
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

print("\nInvocando el trimmer\n")
print(trimmer.invoke(messages))


Invocando el trimmer

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}), HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}), AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}), HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}), AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]


To use it in our chain, we just need to run the trimmer before we pass the messages input to our prompt.

In [41]:
newworkflow = StateGraph(state_schema=State)


def call_model_trimmed(state: State):
    # Invoke the trimmer with the message of the state
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        # Passing the trimmed messages as messages in the prompt template
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


newworkflow.add_edge(START, "model_trimmed")
newworkflow.add_node("model_trimmed", call_model_trimmed)

memory = MemorySaver()
app = newworkflow.compile(checkpointer=memory)

Ahora, si intentamos preguntarle nuestro nombre al modelo, no lo sabrá porque hemos recortado esa parte del historial de chat:

In [42]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't have any information about your name. You didn't tell me, and I'm just a text-based AI assistant, I don't have the ability to store or recall personal data. Each time you interact with me, it's a new conversation! Would you like to tell me your name?


Pero si le preguntamos por información que está dentro de los últimos mensajes, recuerda:

In [43]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked the classic "2 + 2" problem!


## Streaming

Ahora tenemos un chatbot funcionando. Sin embargo, una consideración clave para la experiencia de usuario (UX) en aplicaciones de chatbot es la transmisión. Los LLM a veces tardan en responder, por lo que, para mejorar la experiencia del usuario, la mayoría de las aplicaciones transmiten cada token a medida que se genera. Esto permite al usuario ver el progreso.

¡Es realmente muy fácil!

Por defecto, .stream en nuestra aplicación LangGraph transmite los pasos de la aplicación; en este caso, el paso de la respuesta del modelo. Configurar stream_mode="messages" nos permite transmitir los tokens de salida:

In [44]:
print("\nStreaming\n")
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")


Streaming

Nice| to| meet| you|,| Todd|!| Here|'s| one|:

|What| do| you| call| a| fake| nood|le|?

|(wait| for| it|...)

|An| imp|asta|!

|Hope| that| made| you| laugh|,| Todd|!| Do| you| want| another| one|?||